In [ ]:
"""

Multi-Scale Relief Model (MSRM) Python code developed by H.A. Orengo
to accompany the paper:

Orengo, H.A. and Petrie, C.A. 2018. 'Multi-Scale Relief Model (MSRM): a new algorithm
for the analysis of subtle topographic change in digital elevation models'
published in Earth Surface Processes and Landforms, 43(6): 1361-1369.

Please, refer to the paper for details on the method and how to best use it

"""

import numpy as np
from osgeo import gdal
from scipy.ndimage import convolve
from scipy.ndimage import uniform_filter
from google.colab import drive

# This will mount your Google Drive, it will prompt you for authorization.
# If you do not plan to use your Google Drive, just delete the followng line
drive.mount('/content/drive')

# Add the directory or address with your DSM in your Google Drive here
# The use of Google Drive is optional here you can use your local drive instead
# just provide the address
dsm_file = '/content/drive/MyDrive/proof.tif'

# Read the DSM raster file
dsm_dataset = gdal.Open(dsm_file)
dsm_band = dsm_dataset.GetRasterBand(1)
dsm_array = dsm_band.ReadAsArray()

# Prompt user for maximum and minimum feature size and scaling factor
fmax = int(input('What is maximum size of the feature you want to detect in meters? '))
fmin_prompt = int(input('What is minimum size of the feature you want to detect in meters? '))
x = int(input('Which scaling factor do you want to use? '))

# Calculate the raster resolution (rr) - assuming equal x and y resolution
gt = dsm_dataset.GetGeoTransform()
rr = round((gt[1])*1000) / 1000  # gt[1] is pixel width, gt[5] is pixel height (both in "map" units)

# Check for minimum feature size
fmin = rr if fmin_prompt <= rr else fmin_prompt

# Calculation of 'i' and 'n' values
i = int(((fmin-rr)/(2*rr))** (1/x))
n = int(((fmax-rr)/(2*rr))** (1/x))

# Create a list to store the filtered DSM arrays
filtered_arrays = []

# Apply a uniform_filter to the DSM for each value of ndx from i to n
for ndx in range(i, n + 1):
    size = ndx**x
    filtered_dsm = uniform_filter(dsm_array, size=size)
    filtered_arrays.append(filtered_dsm)

# Subtract consecutive filtered DSM arrays and store the results
difference_arrays = [np.subtract(filtered_arrays[i], filtered_arrays[i+1]) for i in range(len(filtered_arrays) - 1)]

# Calculate the average of all difference arrays
average_difference = np.mean(difference_arrays, axis=0)

# Round to three decimal places
average_difference_rounded = np.round(average_difference, 3)

# Save the result as a new GeoTIFF file in Google Drive
driver = gdal.GetDriverByName('GTiff')
output_path = '/content/drive/MyDrive/MSRM.tif' # Provide a local address if you prefer not to use your Google Drive
out_dataset = driver.Create(output_path, dsm_dataset.RasterXSize, dsm_dataset.RasterYSize, 1, gdal.GDT_Float32)
out_dataset.SetGeoTransform(dsm_dataset.GetGeoTransform())
out_dataset.SetProjection(dsm_dataset.GetProjection())
out_band = out_dataset.GetRasterBand(1)
out_band.WriteArray(average_difference_rounded)
out_dataset = None  # Close the file

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
What is maximum size of the feature you want to detect in meters? 39
What is minimum size of the feature you want to detect in meters? 2
Which scaling factor do you want to use? 2
